In [ ]:
setwd("/Users/leongrinis/Desktop/Massachusetts/")
install.packages("ggplot2")
install.packages("tidyverse")
install.packages("cowplot")
library(sf)
library(tmap)
library(terra)
library(ggplot2)
library(tidyverse)
library(grid)
library(magrittr)
library(cowplot)

New_England <- read_sf ("NEWENGLAND_POLY.shp")
New_England <- New_England[-6, ]

state_abbreviations <- c(
  "NEW HAMPSHIRE" = "NH",
  "VERMONT" = "VT",
  "MASSACHUSETTS" = "MA",
  "CONNECTICUT" = "CT",
  "RHODE ISLAND" = "RI",
  "MAINE" = "ME"
)


New_England$NAME <- state_abbreviations[New_England$NAME]

Existing_windmills <- read_sf("Existing_windmills.geojson")
Massachusetts <- read_sf("massachusetts.geojson" )
windpower <- read_sf ("WINDPOWERGRID50_POLY.shp")
stateline <- read_sf ("OUTLINE25K_ARC.shp")
seafloor <- read_sf ("BTHOS250_POLY.shp")
seafloor_notland <- seafloor[seafloor$DEPTH_NOS != 1, ]

ocean_depth_abbreviations <- c(
  "-15m to -25m" = "-15--25",
  "-25m to -40m" = "-25--40",
  "-40m to -90m" = "-40--90",
  "-90m to -200m" = "-90--200",
  "Below -200m" = "-200--300"
)
seafloor_notland$DEPTHRANGE <- ocean_depth_abbreviations[seafloor_notland$DEPTHRANGE]


Current_windmills <- st_intersection(Existing_windmills, Massachusetts)
optimal_wind_farm_placements <- st_intersection(windpower[windpower$PWRDENSITY > 600, ],
                                                seafloor[seafloor$DEPTH_NOS >= 2 & seafloor$DEPTH_NOS <= 4, ])
latitude <- 41.03 
longitude <- -70.616 
V_point_data <- data.frame(ID = 1, Longitude = longitude, Latitude = latitude)
Vineyard1 <- st_as_sf(V_point_data, coords = c("Longitude", "Latitude"), crs = 4326)


tmap_mode("plot")
  tm_layout(legend.outside = FALSE, legend.position = c("right", "top"), legend.bg.color = "white", legend.text.size = 0.6)+
  tm_shape(seafloor_notland) +
  tm_fill(col = "DEPTHRANGE", palette = "Blues", 
  breaks = c(-300, -200, -90 , -40, -25, -15, 0), 
  labels = c("-15 - -25(m)","-40 - -25(m)","-40 - -90(m)","-90 - -200(m)","-200+ (m)"),  
  title = "Oceanography") +
  tm_borders()+
  tm_shape(windpower) +
  tm_fill(col = "PWRDENSITY", palette = "Reds",
  breaks = c(0, 100, 200, 300, 400, 500, 600, 800, 5800), 
  labels = c("1-100 (watts/sq m)", "100-200(watts/sq m)", "200-300(watts/sq m)", "300-400(watts/sq m)", 
                       "400-500(watts/sq m)", "500-600(watts/sq m)", "600-800(watts/sq m)", "800+(watts/sq m)"), 
  title = "Wind Power") +
  tm_borders()+
  tm_shape(New_England)+
  tm_borders()+
  tm_text(text = "NAME", shadow = TRUE, bg.color="white", size = 0.6, col = "black", root = 0, showNA = FALSE)+
  tm_shape(Massachusetts)+
  tm_borders()+
  tm_shape(stateline) +
  tm_lines()+
  tm_shape(Current_windmills)+
  tm_dots(size=0.20, col="blue")+
  tm_shape(optimal_wind_farm_placements)+
  tm_borders(lwd = 2, lty = "dashed", col = "green") +
  tm_scale_bar(position = c("LEFT", "BOTTOM"), bg.color = "white")+
  tm_compass(north = 0,type = "4star", text.size = 1.2, size = 2, show.labels = 1, cardinal.directions = c("N", "E", "S", "W"), position = c("LEFT", "TOP"))+
  tm_add_legend(type = c("symbol"), labels = "Existing Windfarms",col = "blue",size = 0.5)+
  tm_add_legend(type = c("line"), labels = "Optimal Coastal Windfarms",col = "green",size = 0.5)+
  tm_shape(Vineyard1)+
  tm_dots(size=0.20, col="purple")+
  tm_add_legend(type = c("symbol"), labels = "Offshore Wind Project",col = "purple",size = 0.5)
  
  
  